In [ ]:
from google.colab import drive
import os
import sys
import pandas as pd
from math import log
import pickle as pkl

class Point:
    def __str__(self):
        return "<{}:{}>".format(self.label, self.values)
    def __repr__(self):
        return "<{}:{}>".format(self.label, self.values)
    def __init__(self, label, values):
        self.label = label
        self.values = values

def load_data_from_csv(file_path):
    data = pd.read_csv(file_path)
    points = [Point(row['Class'], row.drop(['Time','Class']).values.tolist()) for _, row in data.iterrows()]
    return points

class Tree:
    def __init__(self):
        self.leaf = True
        self.prediction = None
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None

def predict(tree, point):
    if tree.leaf:
        return tree.prediction
    i = tree.feature
    if point.values[i] < tree.threshold:
        return predict(tree.left, point)
    else:
        return predict(tree.right, point)

def most_likely_class(prediction):
    labels = list(prediction.keys())
    probs = list(prediction.values())
    return labels[probs.index(max(probs))]

def accuracy(data, predictions):
    total = 0
    correct = 0
    for i in range(len(data)):
        point = data[i]
        pred = predictions[i]
        guess = most_likely_class(pred)
        if guess == point.label:
            correct += 1
        total += 1
    return float(correct) / total if total > 0 else 0.0

def precision_recall_f1(data, predictions, positive_label=1):
    tp = 0
    fp = 0
    fn = 0
    for i in range(len(data)):
        point = data[i]
        pred = predictions[i]
        guess = most_likely_class(pred)
        if guess == positive_label and point.label == positive_label:
            tp += 1
        elif guess == positive_label and point.label != positive_label:
            fp += 1
        elif guess != positive_label and point.label == positive_label:
            fn += 1

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    return precision, recall, f1

def split_data(data, feature, threshold):
    left = []
    right = []
    for point in data:
        if point.values[feature] < threshold:
            left.append(point)
        else:
            right.append(point)
    return left, right

def count_labels(data):
    counts = {}
    for point in data:
        label = point.label
        if label in counts:
            counts[label] += 1
        else:
            counts[label] = 1
    return counts

def counts_to_entropy(counts):
    entropy = 0.0
    total = sum(counts.values())
    for count in counts.values():
        if count == 0:
            continue
        probability = count / total
        entropy -= probability * log(probability, 2)
    return entropy

def get_entropy(data):
    counts = count_labels(data)
    entropy = counts_to_entropy(counts)
    return entropy

def find_best_threshold_fast(data, feature):
    entropy = get_entropy(data)
    best_gain = 0
    best_threshold = None
    data = sorted(data, key=lambda point: point.values[feature])
    total = len(data)
    left = {}
    right = count_labels(data)
    left_total = 0
    right_total = total
    for i in range(total - 1):
        point = data[i]
        label = point.label
        left[label] = left.get(label, 0) + 1
        right[label] -= 1
        left_total += 1
        right_total -= 1
        current = point.values[feature]
        next_value = data[i + 1].values[feature]
        if current == next_value:
            continue
        left_entropy = counts_to_entropy(left)
        right_entropy = counts_to_entropy(right)
        current_entropy = (left_entropy * left_total + right_entropy * right_total) / total
        gain = entropy - current_entropy
        if gain > best_gain:
            best_gain = gain
            best_threshold = next_value
    return best_gain, best_threshold

def find_best_split(data):
    if len(data) < 2:
        return None, None
    best_feature = None
    best_threshold = None
    best_gain = 0
    num_features = len(data[0].values)
    for feature in range(num_features):
        gain, threshold = find_best_threshold_fast(data, feature)
        if gain > best_gain:
            best_gain = gain
            best_feature = feature
            best_threshold = threshold
    return best_feature, best_threshold

def make_leaf(data):
    tree = Tree()
    counts = count_labels(data)
    prediction = {}
    for label in counts:
        prediction[label] = float(counts[label]) / len(data)
    tree.prediction = prediction
    return tree

def c45(data, max_levels):
    if max_levels <= 0:
        return make_leaf(data)
    counts = count_labels(data)
    if len(counts) == 1:
        return make_leaf(data)
    feature, threshold = find_best_split(data)
    if feature is None or threshold is wNone:
        return make_leaf(data)
    left, right = split_data(data, feature, threshold)
    if not left or not right:
        return make_leaf(data)
    tree = Tree()
    tree.leaf = False
    tree.feature = feature
    tree.threshold = threshold
    tree.left = c45(left, max_levels - 1)
    tree.right = c45(right, max_levels - 1)
    return tree

def load_data_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        return pkl.load(f)

file_path = '/content/creditcard.csv'
data = load_data_from_csv(file_path)

# Split dataset
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

# Train
max_depth = 2
tree = c45(train_data, max_depth)
predictions = [predict(tree, point) for point in test_data]

accuracy_score = accuracy(test_data, predictions)
precision_score, recall_score, f1_score = precision_recall_f1(test_data, predictions, positive_label=1)

print('Accuracy:', accuracy_score)
print('Precision:', precision_score)
print('Recall:', recall_score)
print('F1 score:', f1_score)


Accuracy: 0.9962374581939799
Precision (Fraud): 0.6875
Recall (Fraud): 0.7857142857142857
F1 (Fraud): 0.7333333333333334
